In [1]:
import os
import numpy as np
from datetime import datetime

import faster_rcnn.roi_data_layer.roidb as rdl_roidb
from faster_rcnn.datasets.factory import get_imdb
from faster_rcnn.fast_rcnn.config import cfg, cfg_from_file

def log_print(text, color=None, on_color=None, attrs=None):
    if cprint is not None:
        cprint(text, color=color, on_color=on_color, attrs=attrs)
    else:
        print(text)



# hyper-parameters
# ------------
imdb_name = 'sunrgbd_train_rgb_i_100_8bits' #train
cfg_file = 'experiments/cfgs/faster_rcnn_end2end_sunrgbd.yml'
pretrained_model = 'data/pretrained_model/VGG_imagenet.npy'
output_dir = 'models/sunrgbd_rgb100/'




# load config
cfg_from_file(cfg_file)

# load data
imdb = get_imdb(imdb_name)

<bound method sunrgbd.default_roidb of <faster_rcnn.datasets.sunrgbd.sunrgbd object at 0x7f442b6a8910>>


In [2]:
gt_roidb=imdb.gt_roidb()

sunrgbd_train_rgb_i_100_8bits gt roidb loaded from /c16/THESE.JORIS/datasets/SUNRGBD_pv/faster_rcnn/cache/sunrgbd_train_rgb_i_100_8bits_gt_roidb.pkl


In [3]:
# print "analysing..."
myClassesDict = {}
for img_rois in gt_roidb:
    for id_cls in img_rois['gt_classes']:
        myClassesDict[imdb._classes[id_cls]] = myClassesDict.get(imdb._classes[id_cls], 0) + 1
        
a = sorted(myClassesDict, key=myClassesDict.get)
for cls in a:
    print cls, myClassesDict[cls]
# print myClassesDict
# print "analysed !"

shelves 1
shower_curtain 11
window 32
bathtub 134
fridge 183
toilet 205
curtain 230
towel 262
bag 269
blinds 326
tv 357
mirror 446
sink 530
bookshelf 703
night_stand 869
paper 1280
books 1397
bed 1512
box 1595
picture 1765
dresser 1862
door 2068
counter 2109
person 2411
whiteboard 2461
sofa 2687
cabinet 3046
lamp 4639
desk 5474
pillow 8988
table 15706
chair 145894


In [4]:
myFreqDict = {}
total = sum(myClassesDict.values())
for cls in myClassesDict.keys():
    myFreqDict[cls]=round(float(myClassesDict[cls])/total,5) + 1e-05
        
b = sorted(myFreqDict, key=myFreqDict.get)
ind_med_freq = (len(b)-1)/2
medianFreq = myFreqDict[b[ind_med_freq]]
for cls in b:
    print cls, myFreqDict[cls]

# myFreqDict

shelves 1e-05
shower_curtain 6e-05
window 0.00016
bathtub 0.00065
fridge 0.00088
toilet 0.00099
curtain 0.00111
towel 0.00126
bag 0.00129
blinds 0.00157
tv 0.00171
mirror 0.00214
sink 0.00254
bookshelf 0.00337
night_stand 0.00416
paper 0.00612
books 0.00668
bed 0.00723
box 0.00763
picture 0.00844
dresser 0.0089
door 0.00988
counter 0.01008
person 0.01152
whiteboard 0.01176
sofa 0.01284
cabinet 0.01455
lamp 0.02216
desk 0.02614
pillow 0.04292
table 0.075
chair 0.69656


In [5]:
myWeightsDict = {}
for cls in myClassesDict.keys():
    myWeightsDict[cls] = 1+round(medianFreq/myFreqDict[cls],3)
    print 'classes_weights_dict[\''+cls+'\'] = '+str(myWeightsDict[cls])

classes_weights_dict['toilet'] = 7.182
classes_weights_dict['whiteboard'] = 1.52
classes_weights_dict['night_stand'] = 2.471
classes_weights_dict['paper'] = 2.0
classes_weights_dict['mirror'] = 3.86
classes_weights_dict['table'] = 1.082
classes_weights_dict['chair'] = 1.009
classes_weights_dict['bag'] = 5.744
classes_weights_dict['tv'] = 4.579
classes_weights_dict['window'] = 39.25
classes_weights_dict['lamp'] = 1.276
classes_weights_dict['curtain'] = 6.514
classes_weights_dict['blinds'] = 4.898
classes_weights_dict['dresser'] = 1.688
classes_weights_dict['picture'] = 1.725
classes_weights_dict['door'] = 1.619
classes_weights_dict['shower_curtain'] = 103.0
classes_weights_dict['fridge'] = 7.955
classes_weights_dict['cabinet'] = 1.421
classes_weights_dict['sink'] = 3.409
classes_weights_dict['desk'] = 1.234
classes_weights_dict['bookshelf'] = 2.816
classes_weights_dict['box'] = 1.802
classes_weights_dict['towel'] = 5.857
classes_weights_dict['shelves'] = 613.0
classes_weights_dict['sofa

In [6]:
_classes = ('__background__', # always index 0
                         #'wall',
                        #'floor',
                        'cabinet',
                        'bed',
                        'chair',
                        'sofa',
                        'table',
                        'door',
                        'window',
                        'bookshelf',
                        'picture',
                        'counter',
                        'blinds',
                        'desk',
                        'shelves',
                        'curtain',
                        'dresser',
                        'pillow',
                        'mirror',
                        #'floor_mat',
                        #'clothes',
                        #'ceiling',
                        'books',
                        'fridge',
                        'tv',
                        'paper',
                        'towel',
                        'shower_curtain',
                        'box',
                        'whiteboard',
                        'person',
                        'night_stand',
                        'toilet',
                        'sink',
                        'lamp',
                        'bathtub',
                        'bag')
_class_to_ind = dict(zip(_classes, xrange(len(_classes))))

In [7]:
weights = [-1]
for i in range(1,len(_classes)):
    print _classes[i],myWeightsDict[_classes[i]]
    weights= weights+[myWeightsDict[_classes[i]]]
print weights

cabinet 1.421
bed 1.846
chair 1.009
sofa 1.477
table 1.082
door 1.619
window 39.25
bookshelf 2.816
picture 1.725
counter 1.607
blinds 4.898
desk 1.234
shelves 613.0
curtain 6.514
dresser 1.688
pillow 1.143
mirror 3.86
books 1.916
fridge 7.955
tv 4.579
paper 2.0
towel 5.857
shower_curtain 103.0
box 1.802
whiteboard 1.52
person 1.531
night_stand 2.471
toilet 7.182
sink 3.409
lamp 1.276
bathtub 10.415
bag 5.744
[-1, 1.421, 1.846, 1.009, 1.4769999999999999, 1.082, 1.619, 39.25, 2.816, 1.725, 1.607, 4.898, 1.234, 613.0, 6.514, 1.688, 1.143, 3.86, 1.916, 7.955, 4.579000000000001, 2.0, 5.857, 103.0, 1.802, 1.52, 1.5310000000000001, 2.471, 7.182, 3.409, 1.276, 10.415, 5.744]


In [8]:
len(_classes)

33

In [9]:
len(weights)

33